# Relatório do Projeto Final

## Machine Learning - 2023.1

### Autores: Marco Moliterno e Renato Falcão

Este relatório tem o objetivo de descrever a aplicação de técnicas de aprendizado de máquina para prever o preço de cartas de Yu-Gi-Oh baseando-se nas múltiplas características das mesmas.

O dataset estudado é uma união da base de dados, disponibilizada no [Kaggle](https://www.kaggle.com/), com os preços médios das cartas, disponibilizada pela API do website [Yu-Gi-Oh Prices](https://yugiohprices.com/).

#### Referências

Base de dados do Kaggle disponível [aqui](https://www.kaggle.com/datasets/thedevastator/yu-gi-oh-dataset?select=yugioh_enriched.csv).

Requisições web realizadas ao iterar sobre os sets de cartas, obtendo os preços das cartas pertencentes aos respectivos sets. Esta API pode ser consumida pelo endpoint `https://yugiohprices.com/api/set_data/{nome_do_set}`. Exemplo de resposta pode ser observado [aqui](https://yugiohprices.com/api/set_data/2013%20Collectible%20Tins%20Wave%201).

## Inicio do projeto

### Importando bibliotecas e carregando base de dados

Para realizar o estudo, estaremos utilizando especialmente as bibliotecas *Pandas*, *MatPlotLib*, *Scikit-Learn* e *Numpy*. Vamos importá-las e carregar a base de dados com o Pandas.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

df = pd.read_csv("dbs/yugi_db_polished.csv", index_col=0)

df.head()

,Name,Rarity,Price,Description,CardType,Attribute,ATK,DEF,LVL,Property,MonsterType,isEffect,Duelist,Ability1,Ability2,isTuner,isPendulum
0,XX-Saber Boggart Knight,Shatterfoil Rare,2.73,When this card is Normal Summoned: You can Spe...,monster,earth,1900.0,1000.0,4.0,NaN,Beast-Warrior,Effect,NaN,NaN,NaN,NaN,NaN
1,Gagaga Cowboy,Shatterfoil Rare,4.68,2 Level 4 monsters\r\n\r\nOnce per turn: You c...,monster,earth,1500.0,2400.0,4.0,NaN,Warrior,Effect,NaN,Xyz,NaN,NaN,NaN
2,Forbidden Chalice,Shatterfoil Rare,2.09,Target 1 face-up monster on the field; until t...,spell,NaN,NaN,NaN,NaN,Quick-Play,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Fairy Cheer Girl,Shatterfoil Rare,2.06,2 Level 4 Fairy-Type monsters\r\n\r\nYou can d...,monster,light,1900.0,1500.0,4.0,NaN,Fairy,Effect,NaN,Xyz,NaN,NaN,NaN
4,Exploder Dragon,Shatterfoil Rare,2.08,If this card is destroyed by battle and sent t...,monster,earth,1000.0,0.0,3.0,NaN,Dragon,Effect,NaN,NaN,NaN,NaN,NaN


### Análise Exploratória e Limpeza dos Dados

Com os dados em mãos, podemos começar realizando uma análise exploratória na qual desejamos identificar as características do dataset, observando a estrutura dos dados disponíveis, presença de anomalias, tendências, padrões, distribuições e dados ausentes. O entendimento dos dados é fundamental para se poder tomar decisões com bom embasamento acerca da abordagem que utilizaremos.

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11574 entries, 0 to 11573
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Name         11574 non-null  object 
 1   Rarity       11574 non-null  object 
 2   Price        11574 non-null  float64
 3   Description  11574 non-null  object 
 4   CardType     11574 non-null  object 
 5   Attribute    7470 non-null   object 
 6   ATK          7580 non-null   float64
 7   DEF          7580 non-null   float64
 8   LVL          7580 non-null   float64
 9   Property     4096 non-null   object 
 10  MonsterType  7478 non-null   object 
 11  isEffect     7478 non-null   object 
 12  Duelist      102 non-null    object 
 13  Ability1     1696 non-null   object 
 14  Ability2     137 non-null    object 
 15  isTuner      453 non-null    object 
 16  isPendulum   277 non-null    object 
dtypes: float64(4), object(13)
memory usage: 1.6+ MB


In [3]:
print(df.columns)

Index(['Name', 'Rarity', 'Price', 'Description', 'CardType', 'Attribute',
       'ATK', 'DEF', 'LVL', 'Property', 'MonsterType', 'isEffect', 'Duelist',
       'Ability1', 'Ability2', 'isTuner', 'isPendulum'],
      dtype='object')


**Name**: O nome da carta. Cada carta de Yu-Gi-Oh tem um nome único que a distingue de outras cartas.

**Rarity**: Refere-se à frequência com que a carta aparece em pacotes de cartas. As cartas raras são mais difíceis de encontrar do que as comuns. Exemplos de raridades incluem "Common", "Rare", "Ultra Rare", "Secret Rare", entre outros.

**Price**: O preço de mercado da carta. Isso pode variar com base em vários fatores, como raridade, demanda, condição da carta e se ela está em circulação.

**Description**: Uma descrição textual da carta que geralmente inclui suas habilidades, efeitos ou lore.

**CardType**: O tipo de carta. As cartas de Yu-Gi-Oh podem ser de vários tipos, como Monstros, Magias, ou Armadilhas.

**Attribute**: Este é um traço específico dos monstros. Existem 6 atributos principais: Luz, Trevas, Fogo, Água, Terra e Vento.

**ATK**: Abreviação de "Attack Points". Este é o poder de ataque de uma carta de monstro. Quando dois monstros batalham, o monstro com o ATK mais alto geralmente destrói o monstro com o ATK mais baixo.

**DEF**: Abreviação de "Defense Points". Este é o poder de defesa de uma carta de monstro. É usado quando o monstro está em posição de defesa.

**LVL**: Abreviação de "Level". Este é o nível de uma carta de monstro. O nível de um monstro geralmente determina seu poder, e também é usado para o mecanismo de invocação de tributo.

**Property**: Este é um atributo de cartas de Magia e Armadilha. As propriedades de cartas de Magia incluem "Normal", "Contínua", "Equipamento", "Campo", "Rápida", etc. As cartas de Armadilha têm propriedades "Normal", "Contínua" e "Contra".

**MonsterType**: Isto é específico para cartas de monstros e refere-se à sua classificação. Exemplos incluem "Guerreiro", "Besta", "Dragão", "Demoníaco", etc.

**isEffect**: Um indicador de se o monstro é uma carta de Monstro de Efeito. Estes monstros têm habilidades especiais que são ativadas sob certas condições.

**Duelist**: O duelista a qual a Skill pertence 

**Ability1** e **Ability2**: Estes se referem a habilidades especiais ou efeitos que a carta pode ter. Algumas cartas podem ter várias habilidades.

**isTuner**: Um indicador de se a carta é um Monstro Tuner. Estes monstros são usados para Invocações Sincro.

**isPendulum**: Um indicador de se a carta é um Monstro de Pêndulo. Estes monstros podem ser invocados usando a mecânica de Invocação por Pêndulo. Além disso, eles têm escalas de Pêndulo que permitem a Invocação de vários monstros.

É possível notar que diversas colunas têm uma grande quantidade de dados faltantes. No entanto, esses dados "faltantes" significam, na verdade, se as cartas possuem certos atributos ou não. Para analisar as características individuais de cada tipo de carta ("CardType"), podemos analisar um recorte de cada um dos tipos:

In [4]:
categoric_card_caracteristics = ["Attribute", "Property", "MonsterType", "isEffect", "Duelist", "Ability1", "Ability2", "isTuner", "isPendulum"]
numeric_card_caracteristics = ["ATK", "DEF", "LVL"]

card_types = df["CardType"].unique()

for card_type in card_types:
    print(f"Card Type: {card_type}")
    for caracteristica in categoric_card_caracteristics:
        recorte = df[df["CardType"] == card_type]
        print(f"{caracteristica}: {recorte[caracteristica].unique()}")
    print("\n")

Card Type: monster
Attribute: ['earth' 'light' 'water' 'dark' 'wind' 'divine' 'fire' nan '?']
Property: [nan]
MonsterType: ['Beast-Warrior' 'Warrior' 'Fairy' 'Dragon' 'Aqua' 'Zombie' 'Rock' 'Beast'
 'Insect' 'Sea Serpent' 'Fiend' 'Reptile' 'Plant' 'Thunder' 'Spellcaster'
 'Machine' 'Winged Beast' 'Divine-Beast' 'Dinosaur' 'Pyro' 'Fish' 'Normal'
 'Psychic' 'Cyberse' 'Wyrm' 'Token']
isEffect: ['Effect' 'Normal']
Duelist: [nan]
Ability1: [nan 'Xyz' 'Fusion' 'Synchro' 'Ritual' 'Toon' 'Link' 'Union' 'Gemini']
Ability2: [nan 'Flip' 'Spirit']
isTuner: [nan 'Tuner']
isPendulum: [nan 'Pendulum']


Card Type: spell
Attribute: [nan]
Property: ['Quick-Play' 'Normal' 'Equip' 'Field' 'Continuous' 'Ritual']
MonsterType: [nan]
isEffect: [nan]
Duelist: [nan]
Ability1: [nan]
Ability2: [nan]
isTuner: [nan]
isPendulum: [nan]


Card Type: trap
Attribute: [nan]
Property: ['Normal' 'Counter' 'Continuous']
MonsterType: [nan]
isEffect: [nan]
Duelist: [nan]
Ability1: [nan]
Ability2: [nan]
isTuner: [nan]
isPendu

Desta forma, é possível inferir a seguinte relação dos tipos de cartas para as características que podem possuir:

- **Monster**: Possui *Attribute*, *ATK*, *DEF*, *LVL*, *MonsterType*, *isEffect*, *Ability1*, *Ability2*, *isTuner* e *isPendulum*;
- **Spell**: Possui apenas *Property*;
- **Trap**: Possui apenas *Property*;
- **Skill**: Possui *Property* e *Duelist*;

No entanto, podemos perceber que existem cartas do tipo **Monster** cujo *Attribute* é "NaN" e "?", ou seja, são valores nulos. Observando esses valores, obtemos o seguinte:

In [5]:
anomalias_nan = df[df["CardType"] == "monster"][df["Attribute"].isna() == True]
anomalias_nan

C:\Users\renat\AppData\Local\Temp\ipykernel_12724\321881468.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  anomalias_nan = df[df["CardType"] == "monster"][df["Attribute"].isna() == True]


,Name,Rarity,Price,Description,CardType,Attribute,ATK,DEF,LVL,Property,MonsterType,isEffect,Duelist,Ability1,Ability2,isTuner,isPendulum
209,Token,Common,4.23,This card can be used as any Token.,monster,NaN,0.0,0.0,0.0,NaN,Normal,Normal,NaN,NaN,NaN,NaN,NaN
4195,Ecclesia the Exiled,Common,1.21,This card can be used as any Token or Counter....,monster,NaN,0.0,0.0,0.0,NaN,Token,Normal,NaN,NaN,NaN,NaN,NaN
4206,Albaz the Shrouded,Common,1.21,This card can be used as any Token or Counter....,monster,NaN,0.0,0.0,0.0,NaN,Token,Normal,NaN,NaN,NaN,NaN,NaN
4209,Aluber the Dogmatic,Common,1.25,This card can be used as any Token or Counter....,monster,NaN,0.0,0.0,0.0,NaN,Token,Normal,NaN,NaN,NaN,NaN,NaN
4229,The Virtuous Vestals,Common,1.21,This card can be used as any Token or Counter....,monster,NaN,0.0,0.0,0.0,NaN,Token,Normal,NaN,NaN,NaN,NaN,NaN
4233,Tri-Brigade,Common,1.22,This card can be used as any Token or Counter....,monster,NaN,0.0,0.0,0.0,NaN,Token,Normal,NaN,NaN,NaN,NaN,NaN
9369,Jesse Anderson - Bonder with the Crystal Beasts,Super Rare,1.25,This card can be used as any Token or Counter....,monster,NaN,0.0,0.0,0.0,NaN,Normal,Normal,NaN,NaN,NaN,NaN,NaN
9370,Jesse and Ruby - Unleashing the Legend,Super Rare,1.25,This card can be used as any Token or Counter....,monster,NaN,0.0,0.0,0.0,NaN,Normal,Normal,NaN,NaN,NaN,NaN,NaN


In [6]:
anomalias_interrogacao = df[df["CardType"] == "monster"][df["Attribute"] == "?"]
anomalias_interrogacao

C:\Users\renat\AppData\Local\Temp\ipykernel_12724\4057386446.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  anomalias_interrogacao = df[df["CardType"] == "monster"][df["Attribute"] == "?"]


,Name,Rarity,Price,Description,CardType,Attribute,ATK,DEF,LVL,Property,MonsterType,isEffect,Duelist,Ability1,Ability2,isTuner,isPendulum
7801,Crystal Beast Token,Common,1.18,"This card can be used as a ""Crystal Beast Toke...",monster,?,0.0,0.0,0.0,NaN,Normal,Normal,NaN,NaN,NaN,NaN,NaN
10216,Duel Dragon Token,Super Rare,1.84,"This card can be used as a ""Duel Dragon Token""...",monster,?,0.0,0.0,0.0,NaN,Normal,Normal,NaN,NaN,NaN,NaN,NaN
10464,Option Token,Super Rare,1.48,"This card can be used as an ""Option Token"".\r\...",monster,?,0.0,0.0,0.0,NaN,Normal,Normal,NaN,NaN,NaN,NaN,NaN


Essas cartas na verdade são **Tokens** do jogo, que são representações de **Monstros**. Desta forma, não vamos considerá-las para a análise.

In [7]:
df.drop(anomalias_nan.index, inplace=True)
df.drop(anomalias_interrogacao.index, inplace=True)

In [10]:
df[df["MonsterType"] == "Normal"]

,Name,Rarity,Price,Description,CardType,Attribute,ATK,DEF,LVL,Property,MonsterType,isEffect,Duelist,Ability1,Ability2,isTuner,isPendulum


Agora, a ideia é realizar um pré-processamento desses dados ...

### Pré-processamento dos dados

Nesta etapa, os dados são transformados em uma forma adequada para serem utilizados pelos modelos de Machine Learning. Isso pode envolver a normalização ou padronização das variáveis, a codificação de variáveis categóricas, a redução de dimensionalidade e outras técnicas de preparação dos dados.

### Text Embedding

A informação da descrição ("Description") das cartas, apesar de ser um texto complexo e não apenas uma variável categórica ou numérica, ainda pode ser transformado em informação útil para o modelo. A solução é utilizar um modelo pré-treinado do Keras para obter um vetor numérico a partir das palavras da descrição. Com este vetor, é possível realizar uma decomposição conhecida com *Principal Component Analysis* (ou *PCA*), reduzindo este vetor à apenas 5 números, que serão passados como parâmetros para o modelo de regressão.

In [ ]:
import tensorflow_hub as hub
from sklearn.decomposition import PCA

model = hub.KerasLayer("https://tfhub.dev/google/nnlm-en-dim128/2")

DATAFRAME = df.copy()

In [ ]:
embeddings = model(DATAFRAME["text"])
embeddings.shape

In [ ]:
pca_model = PCA(n_components=5)
pca_DF = pca_model.fit_transform(embeddings)
pca_DF

In [ ]:
df_new = pd.DataFrame(pca_DF, columns=['pca1', 'pca2','pca3','pca4','pca5'])
df_new

In [ ]:
df1 = DATAFRAME.copy()
df1 = df1.join(df_new)
df1

### Divisão dos dados

Antes de prosseguir para a seleção de um modelo e treinamento, é necessário dividir os dados em conjuntos de treinamento, validação e teste. O conjunto de treinamento é utilizado para treinar o modelo, o conjunto de validação é usado para ajustar os parâmetros do modelo e o conjunto de teste é utilizado para avaliar o desempenho final do modelo. Esta divisão é importante para evitar que a avaliação do modelo seja enviesada pelo resultado final, de forma a "overfittar" para os dados de teste. 

### Seleção e Treinamento do modelo

O modelo selecionado é o ... e será treinado usando o conjunto de treinamento. O treinamento envolve alimentar os dados ao modelo e ajustar seus parâmetros com o objetivo de aprender os padrões presentes nos dados.

### Avaliação do modelo

O desempenho do modelo é avaliado usando o conjunto de validação. Métricas como acurácia, precisão, recall, F1-score e curvas de aprendizado são utilizadas para avaliar o quão bem o modelo está generalizando os dados.

### Ajuste do modelo

Com base na avaliação do modelo, é possível ajustar os parâmetros, selecionar diferentes algoritmos ou fazer outras modificações para melhorar o desempenho do modelo.

### Teste do modelo

O modelo ajustado é então testado usando o conjunto de teste, que contém dados não vistos anteriormente. Isso permite avaliar como o modelo se comporta em situações reais e verificar sua capacidade de generalização.